In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime
import matplotlib.pyplot as plt
from pprint import pprint

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")

2023-05-31 15:51:03.698400: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-31 15:51:03.730281: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-31 15:51:03.731784: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 15:51:04.505756: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset = pd.read_csv("../StationDateDatasets/datasetStationDate.csv").drop(columns=["Unnamed: 0"])

In [3]:
cols = ["StationEnd", "WorkingDay", "Hour", "Count", "Count1", "Count1week", "Count2week", "Count3week", "Temp", "ATemp", "WeatherSituation"]

epochs = 30
batch_size = 1
verbose = 2

In [4]:
met1 = dataset[[col for col in dataset.columns if col in cols]]

X = met1.drop(columns=["Count"])
y = met1["Count"]

X = np.reshape(X.values, (X.shape[0], 1, X.shape[1]))

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=False)

acc_per_fold = []
loss_per_fold = []

model_met1 = Sequential()
model_met1.add(LSTM(4, input_shape=(1,met1.shape[1]-1)))
model_met1.add(Dense(1))
model_met1.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])

# K-fold Cross Validation model evaluation
for train, test in kfold.split(X, y):
    model_met1.fit(X[train], y[train], epochs=epochs, batch_size=batch_size, verbose=verbose)

    # Generate generalization metrics
    scores = model_met1.evaluate(X[test], y[test], verbose=0)
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
mean = np.mean(loss_per_fold)
print(f'> Loss: {mean}')
print('------------------------------------------------------------------------')

2023-05-31 16:50:34.427661: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-31 16:50:34.428334: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/30


2023-05-31 16:50:34.862915: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-31 16:50:34.864847: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-31 16:50:34.865767: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

183062/183062 - 164s - loss: 4.1198 - accuracy: 0.3202 - 164s/epoch - 897us/step
Epoch 2/30
183062/183062 - 159s - loss: 3.9421 - accuracy: 0.3205 - 159s/epoch - 870us/step
Epoch 3/30
183062/183062 - 160s - loss: 3.9173 - accuracy: 0.3208 - 160s/epoch - 873us/step
Epoch 4/30
183062/183062 - 160s - loss: 3.9017 - accuracy: 0.3207 - 160s/epoch - 873us/step
Epoch 5/30
183062/183062 - 160s - loss: 3.8968 - accuracy: 0.3207 - 160s/epoch - 873us/step
Epoch 6/30
183062/183062 - 160s - loss: 3.8893 - accuracy: 0.3207 - 160s/epoch - 872us/step
Epoch 7/30
183062/183062 - 161s - loss: 3.8885 - accuracy: 0.3208 - 161s/epoch - 879us/step
Epoch 8/30
183062/183062 - 157s - loss: 3.8788 - accuracy: 0.3207 - 157s/epoch - 859us/step
Epoch 9/30
183062/183062 - 159s - loss: 3.8728 - accuracy: 0.3207 - 159s/epoch - 869us/step
Epoch 10/30
183062/183062 - 159s - loss: 3.8601 - accuracy: 0.3204 - 159s/epoch - 869us/step
Epoch 11/30
183062/183062 - 159s - loss: 3.8247 - accuracy: 0.3191 - 159s/epoch - 871us/st

2023-05-31 18:10:21.182147: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-31 18:10:21.183550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-31 18:10:21.184837: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/30
183062/183062 - 162s - loss: 3.5883 - accuracy: 0.3335 - 162s/epoch - 882us/step
Epoch 2/30
183062/183062 - 161s - loss: 3.5934 - accuracy: 0.3338 - 161s/epoch - 882us/step
Epoch 3/30
183062/183062 - 162s - loss: 3.5892 - accuracy: 0.3337 - 162s/epoch - 883us/step
Epoch 4/30
183062/183062 - 161s - loss: 3.5856 - accuracy: 0.3336 - 161s/epoch - 880us/step
Epoch 5/30
183062/183062 - 161s - loss: 3.5849 - accuracy: 0.3338 - 161s/epoch - 880us/step
Epoch 6/30
183062/183062 - 161s - loss: 3.5832 - accuracy: 0.3337 - 161s/epoch - 879us/step
Epoch 7/30
183062/183062 - 161s - loss: 3.5860 - accuracy: 0.3338 - 161s/epoch - 879us/step
Epoch 8/30
183062/183062 - 160s - loss: 3.5855 - accuracy: 0.3338 - 160s/epoch - 876us/step
Epoch 9/30
183062/183062 - 161s - loss: 3.5895 - accuracy: 0.3337 - 161s/epoch - 878us/step
Epoch 10/30
183062/183062 - 161s - loss: 3.5805 - accuracy: 0.3338 - 161s/epoch - 880us/step
Epoch 11/30
183062/183062 - 161s - loss: 3.5839 - accuracy: 0.3338 - 161s/epoch